In [1]:
import pandas as pd

In [2]:
clust_to_celltype = {1: 'excitatory_neurons', 2: 'inhibitory_neurons', 3: 'excitatory_neurons',
                     4: 'excitatory_neurons', 5: 'nigral_neurons', 6: 'nigral_neurons', 7: 'unknown_neurons',
                     8: 'opcs', 9: 'opcs', 10: 'opcs', 11: 'inhibitory_neurons',12: 'inhibitory_neurons',
                     13: 'astrocytes', 14: 'astrocytes', 15: 'astrocytes', 16: 'astrocytes', 17: 'astrocytes',
                     18: 'doublets', 19: 'oligodendrocytes', 20: 'oligodendrocytes', 21: 'oligodendrocytes',
                     22: 'oligodendrocytes', 23: 'oligodendrocytes', 24: 'microglia'}

In [3]:
all_scores = pd.read_csv('/oak/stanford/groups/akundaje/projects/alzheimers_parkinsons/updated_snp_scores/idr_snps/all.idr.unique.snps.scores.tsv',
                         sep='\t', keep_default_na=False)
scores_dict = {}
vars_to_check = ['chr','start','end','rsid','effect','noneffect','ref','alt',
                 'major','minor','direction','locus_num','gwas','gwas_pval','coloc',
                 'explain_pval','ism_pval','delta_pval','magnitude_pval','prominence_pval','confidence']
for i in range(1,25):
    scores_dict[i] = pd.read_csv('/oak/stanford/groups/akundaje/projects/alzheimers_parkinsons/updated_snp_scores/idr_snps/Cluster'
                                 + str(i) + '.' + clust_to_celltype[i] + '.updated.idr.snp_scores.tsv', sep='\t', keep_default_na=False)
for index,row in all_scores.iterrows():
    best_df = scores_dict[row['best_cluster']]
    best_scores = best_df.loc[(best_df['rsid'] == row['rsid']) & (best_df['effect'] == row['effect']) & (best_df['noneffect'] == row['noneffect'])]
    if len(best_scores) > 1:
        print('Duplicate:')
        print(row['rsid'])
        print('-----')
    for ind,rw in best_scores.iterrows():
        for var in vars_to_check:
            if isinstance(row[var], float):
                row[var] = round(row[var], 6)
                rw[var] = round(rw[var], 6)
                if row[var] != rw[var]:
                    print(row['rsid'])
                    print(row['best_cluster'])
                    print(var)
                    print(row[var])
                    print(rw[var])
                    print('-----')

rs5004356
19
gwas_pval
0.013934
0.013933
-----
Duplicate:
rs7516590
-----
rs9936329
9
gwas_pval
0.034694
0.034695
-----
rs7527412
20
gwas_pval
1.4e-05
1.3e-05
-----


In [4]:
all_snps = '/oak/stanford/groups/akundaje/projects/alzheimers_parkinsons/snps_final/191121_ld_buddies_table_stage3.tsv'
idr_snps = [i.strip() for i in open('/oak/stanford/groups/akundaje/projects/alzheimers_parkinsons/snps_final/IDR_snps.txt')]

In [5]:
all_snps_df = pd.read_csv(all_snps, sep='\t', keep_default_na=False)
display(all_snps_df.head())
print()
print(all_snps_df.shape)

,chr,pos,r2_with_ld_tag,ld_tag_chr,ld_tag_pos,source_gwas,source,snp_id,locus_num,number_ld_buddies,...,nearest_atac_tissue_broad_tissue_regions,start_broad_tissue_regions,end_broad_tissue_regions,dist_broad_tissue_regions,direct_atac_overlap_single_cell,containing_atac_tissues_single_cell,nearest_atac_tissue_single_cell,start_single_cell,end_single_cell,dist_single_cell
0,10,102207833,0.805869,10,102255522,Nalls_23andMe,LD,10_102207833,1,9,...,ALLO,102225838,102226054,18005.0,False,none,microglia,102213693,102214072,5860.0
1,10,102245653,0.987490,10,102255522,Nalls_23andMe,LD,10_102245653,1,9,...,ALLO,102245836,102246040,183.0,True,"astrocytes,doublets,excitatory_neurons,inhibit...","astrocytes,doublets,excitatory_neurons,inhibit...",102244973,102245978,177.5
2,10,102250385,0.809348,10,102255522,Nalls_23andMe,LD,10_102250385,1,9,...,ALLO,102245836,102246040,4345.0,False,none,opcs,102247774,102248243,2142.0
3,10,102251214,0.809348,10,102255522,Nalls_23andMe,LD,10_102251214,1,9,...,ALLO,102245836,102246040,5174.0,False,none,opcs,102247774,102248243,2971.0
4,10,102255522,1.000000,10,102255522,Nalls_23andMe,Nalls-Chang,10_102255522,1,9,...,ALLO,102245836,102246040,9482.0,False,none,inhibitory_neurons,102255548,102256050,26.0



(13773, 38)


In [6]:
idr_peak = []
all_snps_df_uniq_nona = all_snps_df.loc[(all_snps_df['effect_allele'] != 'NA') & (all_snps_df['noneffect_allele'] != 'NA')]
#all_snps_df_uniq_nona = all_snps_df.copy()
all_snps_df_uniq_nona.sort_values(by='rsid', inplace=True)
all_snps_df_uniq_nona.drop_duplicates(subset='rsid', inplace=True)
for index,row in all_snps_df_uniq_nona.iterrows():
    if row['rsid'] in idr_snps:
        idr_peak.append(True)
    else:
        idr_peak.append(False)
all_snps_df_uniq_nona['in_idr_peak'] = idr_peak
display(all_snps_df_uniq_nona.head())
print()
print(all_snps_df_uniq_nona.shape)

/users/soumyak/miniconda3/envs/dragonn/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/users/soumyak/miniconda3/envs/dragonn/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/users/soumyak/miniconda3/envs/dragonn/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#inde

,chr,pos,r2_with_ld_tag,ld_tag_chr,ld_tag_pos,source_gwas,source,snp_id,locus_num,number_ld_buddies,...,start_broad_tissue_regions,end_broad_tissue_regions,dist_broad_tissue_regions,direct_atac_overlap_single_cell,containing_atac_tissues_single_cell,nearest_atac_tissue_single_cell,start_single_cell,end_single_cell,dist_single_cell,in_idr_peak
388,11,47397402,0.965485,11,47372754,Alzheimers_Kunkle_2019,LD,11_47397402,9,220,...,47400574,47400774,3172.0,False,none,microglia,47395347,47395945,1457.0,False
889,11,60164550,1.000000,11,60164550,Alzheimers_Kunkle_2019,GWAS,11_60164550,11,261,...,60130597,60130801,33749.0,False,none,microglia,60183647,60184125,19097.0,False
1084,11,60223118,1.000000,11,60226679,Alzheimers_Kunkle_2019,LD,11_60223118,11,261,...,60265062,60265262,41944.0,False,none,microglia,60221138,60221574,1544.0,False
2089,12,33506496,1.000000,12,33506496,23andme_PD_hg38,GWAS,12_33506496,23,44,...,33529853,33530071,23357.0,False,none,excitatory_neurons,33519350,33521288,12854.0,False
2098,12,33801207,1.000000,12,33801207,23andme_PD_hg38,GWAS,12_33801207,23,44,...,33825726,33825933,24519.0,False,none,inhibitory_neurons,33825550,33826780,24343.0,False



(9542, 39)


In [7]:
all_snps_df_uniq_nona_idr = all_snps_df_uniq_nona.loc[all_snps_df_uniq_nona['in_idr_peak'] == True]
display(all_snps_df_uniq_nona_idr.head())
print()
print(all_snps_df_uniq_nona_idr.shape)

,chr,pos,r2_with_ld_tag,ld_tag_chr,ld_tag_pos,source_gwas,source,snp_id,locus_num,number_ld_buddies,...,start_broad_tissue_regions,end_broad_tissue_regions,dist_broad_tissue_regions,direct_atac_overlap_single_cell,containing_atac_tissues_single_cell,nearest_atac_tissue_single_cell,start_single_cell,end_single_cell,dist_single_cell,in_idr_peak
3146,17,45425634,1.000000,17,45425634,23andme_PD_hg38,GWAS,17_45425634,54,2496,...,45428370,45428573,2736.0,True,microglia,microglia,45425421,45426178,165.5,True
4907,17,45941122,1.000000,17,45941122,23andme_PD_hg38,GWAS,17_45941122,54,2496,...,45942821,45943022,1699.0,True,"astrocytes,oligodendrocytes,opcs","astrocytes,oligodendrocytes,opcs",45940888,45941177,89.5,True
5224,17,46003042,1.000000,17,46003042,23andme_PD_hg38,GWAS,17_46003042,54,2496,...,46003026,46003228,85.0,True,"inhibitory_neurons,microglia,oligodendrocytes","inhibitory_neurons,microglia,oligodendrocytes",46001751,46003162,585.5,True
6761,17,78432505,0.925005,17,78429399,Nalls_23andMe,LD,17_78432505,62,91,...,78432562,78432778,57.0,True,"astrocytes,doublets,oligodendrocytes,opcs","astrocytes,doublets,oligodendrocytes,opcs",78432233,78432735,21.0,True
7255,19,44643846,1.000000,19,44643846,Alzheimers_Kunkle_2019,GWAS,19_44643846,71,855,...,44644177,44644393,331.0,True,"astrocytes,doublets,excitatory_neurons,inhibit...","astrocytes,doublets,excitatory_neurons,inhibit...",44643578,44644825,355.5,True



(1219, 39)
